La función realiza predicciones en todas las imágenes de un directorio especificado (sdir) e imprime los resultados.  
Tiene dos modos de funcionamiento. En el modo no_promedio se hace una predicción separada para cada imagen   
y se imprime la clase y la probabilidad predichas para cada imagen.  
En el modo promediado, se realiza una predicción para cada imagen y se almacenan las probabilidades.  
Cuando se han completado las predicciones de todas las imágenes, se suman las probabilidades de cada clase.    
La clase con la mayor suma de probabilidades se selecciona como la clase predicha para todas las imágenes.  
La clase predicha y su probabilidad promediada se imprimen.  
Supongamos que tenemos 10 imágenes de la cara de un mismo niño y queremos predecir si es autista.    
Puede obtener 10 predicciones individuales y examinar cada una de ellas. Sin embargo, obtendrá una confianza mucho mayor    
utilizando el predictor en el modo de promedio para las 10 imágenes. Por supuesto, se puede predecir con una sola imagen de la cara del niño.  
pero obtendrá una confianza mucho mayor si utiliza varias imágenes faciales del niño y utiliza el predictor en el modo de promedio.
  
 Los parámetros de la función son:   
  * model_path es la ruta completa al modelo entrenado 
  * txt_path es la ruta completa al fichero classes.txt generado por el cuaderno autism.ipynb cuando se ejecuta para producir el modelo entrenado.    
      El archivo classes.txt tiene la forma class0, class1, ...classN. Para el caso del conjunto de datos autista, el archivo aparece como  
      autista,no_autista.
  * img_size es un tupple (height, width) que representa la forma de la imagen utilizada para entrenar el modelo.   
  * mode es una cadena. Si mode=='ave' el predictor proporciona una única predicción de la clase y la probabilidad de clase. Si mode es cualquier    
      otro valor de cadena, el predictor proporciona una predicción individual para cada imagen y devuelve un marco de datos con las columnas  
      nombre de archivo, clase, probabilidad. Cada fila del marco de datos contiene el nombre del archivo de imagen, la clase predicha y la probabilidad    
      de la predicción.

In [1]:
import tensorflow as tf
from tensorflow import keras
import os
import pandas as pd
import numpy as np

In [2]:
def predictor(sdir, model_path, txt_path, img_size,  mode):
    with open(txt_path, 'r') as f:
        content=f.read() 
    split=content.split(',')
    classes=[split[i] for i in range(len(split))]
    num_classes=len(classes) 
    print("Carga del modelo. Puede tardar varios segundos")
    model = tf.keras.models.load_model(model_path)
    print ('Modelo cargado')
    flist=sorted(os.listdir(sdir))
    print ('prediciendo la clase del ', len(flist), ' archivos en  ', sdir)   
    filepaths=[]       
    sum_list=[0 for i in range( num_classes )]    
    for i, f in enumerate(flist):
        fpath=os.path.join(sdir,f)
        filepaths.append(fpath)
    Fseries=pd.Series(filepaths, name ='filepaths')
    df=pd.concat([Fseries], axis=1)
    gen=tf.keras.preprocessing.image.ImageDataGenerator().flow_from_dataframe(df, x_col='filepaths', y_col=None, target_size=img_size, class_mode=None, shuffle=False)
    preds=model.predict(gen)
    cclass=[] # Lista de clases pronosticadas
    pclass=[] # Lista de probabilidades de clase   
    for i, p in enumerate(preds):   # Iterar a través de las predicciones     
        for  j, cp in enumerate(p): # Actualizar la lista_suma añadiendo las probabilidades de cada clase
            sum_list[j] +=cp        
        index=np.argmax(p) # Encontrar el índice del valor en p que tiene la probabilidad más alta
        klass=classes[index] # Obtener la clase con mayor probabilidad
        cclass.append(klass)
        pclass.append(p[index]* 100)
    Pseries=pd.Series(pclass, name='% Probability')
    Cseries=pd.Series(cclass, name='Predicted Class')
    Fseries=pd.Series(flist, name='Filename')                                                                          
    pdf=pd.concat([Fseries, Cseries, Pseries], axis=1)  # marco de datos del form Filename, Predicted Class, % Probability      
    ave_index=np.argmax(sum_list)
    ave_class=classes[ave_index]    
    ave_p=sum_list[ave_index]* 100/len(flist)    
    if mode == 'ave':
        print (f'The class prediction for all images is {ave_class} with a probability of {ave_p:6.2f} %')
        return ave_class, ave_p                                             
    else:
        print (pdf)
        return pdf                                                                   
   

In [3]:
# Ejemplo en modo no promediado: se realizan predicciones individuales y se devuelve un marco de datos
txt_path=r'C:\Temp\Autism\classes.txt'
sdir=r'C:\Temp\Autism\tests_non_autistic'
img_size=(200,200)
mode=''
model_path=r'C:\Temp\Autism\EfficientNetB5-Autism-92.00.h5'
df= predictor(sdir, model_path, txt_path, img_size, mode)

Loading the model. This may take several seconds
model is loaded
predicting the class of the  5  files in  C:\Temp\Autism\tests_non_autistic
Found 5 validated image filenames.
  Filename Predicted Class  % Probability
0  006.jpg    non_autistic      90.747321
1  007.jpg    non_autistic      99.004108
2  008.jpg    non_autistic      87.127084
3  009.jpg        autistic      99.136752
4  010.jpg    non_autistic      99.059314


In [5]:
# example on same data of averaging mode - 1 prediction is made for all images. returns the predicted class and its probability
mode='ave'
ave_class, ave_p  =predictor(sdir, model_path, txt_path, img_size,  mode)

Loading the model. This may take several seconds
model is loaded
predicting the class of the  5  files in  C:\Temp\Autism\tests_non_autistic
Found 5 validated image filenames.
The class prediction for all images is non_autistic with a probability of  75.36 %
